In [49]:
# CREATED: 4-APR-2022
# LAST EDIT: 4-MAY-2022
# AUTHOR(S): DUANE RINEHART
'''
IMPORT POLYGON POINTS FROM CSV (EVENTUALLY PULL FROM RDBMS)
Note: polygons_dump5.csv GENERATED WITH QUERY - SELECT * FROM annotations_points WHERE (prep_id='DK55' AND (label='7N_L' OR label='5N_L')) OR (prep_id='MD594' AND (label='SC' OR label='IC'))
'''
import os
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error

win_data_dir = 'E:/dev/'
linux_data_dir = '/mnt/e/dev/'


'''
SET SRC FOLDER BASED (CROSS-PLATFORM)
'''
if os.name == "nt":
    input_fullPath = os.path.join(win_data_dir)
else:
    input_fullPath = os.path.join(linux_data_dir)

infile = os.path.join(input_fullPath, "MD585_dump5.csv")
outfile = os.path.join(input_fullPath, "output_for_Yoav3.xlsx")

def load_raw_data(infile):
    #return pd.read_csv(infile, sep='\t', index_col=0)

    #DB REVISION - CREDENTIALS IN parameters.yaml
    user = "dklab"
    password = "$pw4dklabdb"
    host = "db.dk.ucsd.edu"
    schema = "active_atlas_development"
    try:
        db_connection = mysql.connector.connect(host=host, user=user, password=password, database=schema)
        if db_connection.is_connected():
            cursor = db_connection.cursor()
    except Error as e:
        print("Error while connecting to MySQL", e)

    sql = "SELECT * FROM annotations_points WHERE (prep_id='DK55' AND (label='7N_L' OR label='5N_L')) OR (prep_id='MD594' AND (label='SC' OR label='IC'))"
    return pd.read_sql(sql, db_connection, index_col=None, coerce_float=True, params=None, parse_dates=None, columns=None, chunksize=None)


def parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns):
    data = df_raw_data[np.logical_and(df_raw_data.prep_id==prep_id,df_raw_data.label==label)]
    data['x'] = data['x'].apply(lambda x: x / xy_resolution_microns_to_pixels)
    data['y'] = data['y'].apply(lambda y: y / xy_resolution_microns_to_pixels)
    data['z'] = data['z'].apply(lambda z: z / z_resolution_section_to_microns)
    data["unique_key"] =  data["prep_id"].astype(str) + '_' + data["label"].astype(str) + '_' + data["z"].round().astype(int).astype(str)
    data["coord"] = list(zip(data['x'], data["y"], data["z"]))
    data[(data['prep_id'] == prep_id) & (data['label'] == label)]
    data = data.rename(columns={'ordering': 'point_ordering'})

    #RECODE, IF NECESSARY
    data.loc[data["FK_structure_id"] == 54, "FK_structure_id"] = "POLYGON"
    data.loc[data["FK_owner_id"] == 4, "FK_owner_id"] = "dk"
    data.loc[data["FK_owner_id"] == 37, "FK_owner_id"] = "beth"
    data.loc[data["FK_input_id"] == 1, "FK_input_id"] = "manual person"


    selected_columns = data[
        ["unique_key", "prep_id", "label", "coord", "z", "polygon_id", "volume_id", "point_ordering"]
    ]  # EXTRACT SPECIFIC COLUMNS FOR ANALYSIS

    # selected_columns = data[
    #     ["unique_key", "prep_id", "label", "coord", "z", "polygon_ordering", 'volume_ordering']
    # ]  # EXTRACT SPECIFIC COLUMNS FOR ANALYSIS

    return selected_columns


df_raw_data = load_raw_data(infile) # REPLACE W/ RDBMS CALL FOR PRODUCTION

# FILTERING CONSTANTS
z_resolution_section_to_microns = 20 # STORED IN DB

# POLYGON SETS BY prep_id, label CREATED INDIVIDUALLY BELOW FOR TESTING
prep_id = 'MD594'
xy_resolution_microns_to_pixels = .452 # STORED IN DB (FOR FOUNDATION BRAINS)
label = 'SC'
extracted_columns_MD594_SC = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)
label = 'IC'
extracted_columns_MD594_IC = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)
prep_id = 'DK55'
xy_resolution_pixels_microns = .325 # STORED IN DB (FOR DK55)
label = '7N_L'
extracted_columns_DK55_7N_L = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)
label = '7N_R'
extracted_columns_DK55_7N_R = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)
label = '5N_L'
extracted_columns_DK55_5N_L = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)
label = '5N_R'
extracted_columns_DK55_5N_R = parse_df(df_raw_data, prep_id, label, xy_resolution_microns_to_pixels, z_resolution_section_to_microns)

# EACH DATAFRAME CONTAINS LIST OF TUPLES (coord) THAT MAY BE USED FOR PLOTTING POLYGONS IN NEUROGLANCER (SOME EXAMPLES IN NOTEBOOK - BELOW)

# FOLLOWING CODE ONLY USEFUL IF BATCH PROCESSING ALL DATA (HOWEVER FOR INTEGRATION IN APP (PIPELINE), LIKELY DB QUERY WITH LOOP IS SUFFICIENT)

consolidated_coordinates = pd.concat([extracted_columns_MD594_SC, extracted_columns_MD594_IC, extracted_columns_DK55_7N_L, extracted_columns_DK55_7N_R, extracted_columns_DK55_5N_L, extracted_columns_DK55_5N_R])

#ENCODE VOLUME ID AS CATEGORICAL VARIABLE
consolidated_coordinates['polygon_ordering'] = pd.factorize(consolidated_coordinates['volume_id'])[0]

# SAVE DATA FOR YOAV INPUTS
consolidated_coordinates.to_excel(outfile, sheet_name='polygons_volumes')








# # CREATE DICTIONARY OF COORDINATES
# transformed_polygon_structures = consolidated_coordinates.groupby(['unique_key'])['coord'].apply(lambda x: list(np.unique(x))).to_dict()
#
# # OUTPUT FORMAT:
# # -KEY IS CONCATENATED prep_id_structure_section ("_" delimiter)
# # -VALUE IS LIST OF COORDINATE TUPLES (x, y, z)
# #
# # TO ACCESS [EXAMPLE WITH prep_id 'MD594', label 'IC', section 140]:
# polygon_index = 'MD594_IC_140'
# for coordinate_list in transformed_polygon_structures[polygon_index]:
#     print(coordinate_list)

<ipython-input-49-a8b443fe09e4>:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['x'] = data['x'].apply(lambda x: x / xy_resolution_microns_to_pixels)
<ipython-input-49-a8b443fe09e4>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['y'] = data['y'].apply(lambda y: y / xy_resolution_microns_to_pixels)
<ipython-input-49-a8b443fe09e4>:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

In [53]:
consolidated_coordinates[(consolidated_coordinates['prep_id'] == 'DK55')]

,unique_key,prep_id,label,coord,z,polygon_id,volume_id,point_ordering,polygon_ordering
39350,DK55_7N_L_164,DK55,7N_L,"(20853.58407079646, 11664.845132743363, 164.0)",164.0,1c530dd8517491c42ffa785fb49cc8e395a2acaf,81ef49231e730c37b56de13b8cbd7e55a6412653,1,2
39351,DK55_7N_L_164,DK55,7N_L,"(20727.455752212387, 11690.95132743363, 164.0)",164.0,1c530dd8517491c42ffa785fb49cc8e395a2acaf,81ef49231e730c37b56de13b8cbd7e55a6412653,2,2
39352,DK55_7N_L_164,DK55,7N_L,"(20701.349557522124, 11743.141592920352, 164.0)",164.0,1c530dd8517491c42ffa785fb49cc8e395a2acaf,81ef49231e730c37b56de13b8cbd7e55a6412653,3,2
39353,DK55_7N_L_164,DK55,7N_L,"(20723.097345132745, 11847.544247787611, 164.0)",164.0,1c530dd8517491c42ffa785fb49cc8e395a2acaf,81ef49231e730c37b56de13b8cbd7e55a6412653,4,2
39354,DK55_7N_L_164,DK55,7N_L,"(20823.141592920354, 11921.504424778761, 164.0)",164.0,1c530dd8517491c42ffa785fb49cc8e395a2acaf,81ef49231e730c37b56de13b8cbd7e55a6412653,5,2
...,...,...,...,...,...,...,...,...,...
41507,DK55_5N_L_160,DK55,5N_L,"(19377.65486725664, 10382.721238938053, 160.0)",160.0,326d9ccc1bc49207042a81164bf1ed34e10f02a6,c3c57cc73d736ac51f98a1291baef909c92f2e23,9,3
41508,DK55_5N_L_160,DK55,5N_L,"(19474.314159292033, 10330.774336283186, 160.0)",160.0,326d9ccc1bc49207042a81164bf1ed34e10f02a6,c3c57cc73d736ac51f98a1291baef909c92f2e23,10,3
41509,DK55_5N_L_160,DK55,5N_L,"(19616.725663716814, 10209.889380530973, 160.0)",160.0,326d9ccc1bc49207042a81164bf1ed34e10f02a6,c3c57cc73d736ac51f98a1291baef909c92f2e23,11,3
41510,DK55_5N_L_160,DK55,5N_L,"(19653.008849557522, 9986.615044247786, 160.0)",160.0,326d9ccc1bc49207042a81164bf1ed34e10f02a6,c3c57cc73d736ac51f98a1291baef909c92f2e23,12,3


In [55]:
consolidated_coordinates['prep_id'].unique()


array(['MD594', 'DK55'], dtype=object)